In [ ]:
%pip install --upgrade git+https://github.com/huggingface/transformers.git 
!sudo apt-get update
!sudo apt-get install -y poppler-utils

In [ ]:
%pip install byaldi  accelerate  pdf2image

In [3]:
from byaldi import RAGMultiModalModel
from transformers import Qwen2VLForConditionalGeneration, AutoTokenizer, AutoProcessor
import torch
import transformers
from transformers import AutoModelForCausalLM, AutoTokenizer


RAG = RAGMultiModalModel.from_pretrained("vidore/colpali")

Verbosity is set to 1 (active). Pass verbose=0 to make quieter.


`config.hidden_act` is ignored, you should use `config.hidden_activation` instead.
Gemma's activation function will be set to `gelu_pytorch_tanh`. Please, use
`config.hidden_activation` if you want to override this behaviour.
See https://github.com/huggingface/transformers/pull/29402 for more details.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
RAG.index(
    input_path="/home/sample.pdf",
    index_name="index",
    store_collection_with_index=False,
    overwrite=True
)

Added page 1 of document 0 to index.
Added page 2 of document 0 to index.
Added page 3 of document 0 to index.
Added page 4 of document 0 to index.
Added page 5 of document 0 to index.
Added page 6 of document 0 to index.
Added page 7 of document 0 to index.
Added page 8 of document 0 to index.
Added page 9 of document 0 to index.
Added page 10 of document 0 to index.
Added page 11 of document 0 to index.
Added page 12 of document 0 to index.
Added page 13 of document 0 to index.
Added page 14 of document 0 to index.
Added page 15 of document 0 to index.
Added page 16 of document 0 to index.
Added page 17 of document 0 to index.
Added page 18 of document 0 to index.
Added page 19 of document 0 to index.
Added page 20 of document 0 to index.
Added page 21 of document 0 to index.
Added page 22 of document 0 to index.
Added page 23 of document 0 to index.
Added page 24 of document 0 to index.
Added page 25 of document 0 to index.
Added page 26 of document 0 to index.
Added page 27 of docu

{0: '/home/sample.pdf'}

In [5]:
from pdf2image import convert_from_path
images = convert_from_path("/home/sample.pdf")

In [ ]:

# # set device
# device = 'cuda'  # or cpu
# torch.set_default_device(device)

device = "cuda"
# create model
model = AutoModelForCausalLM.from_pretrained(
    'BAAI/Bunny-v1_0-4B',
    torch_dtype="auto",
    device_map='auto',
    trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained(
    'BAAI/Bunny-v1_0-4B',
    trust_remote_code=True)

In [17]:
search_query = input("Enter your question ")
results = RAG.search(search_query, k=2)
print(results)
image_indexes = [pages['page_num']-1 for pages in results]
image_indexes

[{'doc_id': 0, 'page_num': 68, 'score': 14.75, 'metadata': {}, 'base64': None}, {'doc_id': 0, 'page_num': 60, 'score': 14.125, 'metadata': {}, 'base64': None}]


[67, 59]

In [18]:
def generate_image_arr(image_indexes):
    image_arr = []
    for i in image_indexes:
        image_arr.append(images[i])
    return image_arr
        


In [23]:

# text = f"Answer the given question from the image if you are unable find the answer from the image then say unable to find the answer. USER: <image>\n{search_query} ASSISTANT:"
# text_chunks = [tokenizer(chunk).input_ids for chunk in text.split('<image>')]
# input_ids = torch.tensor(text_chunks[0] + [-200] + text_chunks[1][1:], dtype=torch.long).unsqueeze(0).to(device)


image_arr = generate_image_arr(image_indexes)
print(image_arr)
# image_tensor = model.process_images([image], model.config).to(dtype=model.dtype, device=device)

text = (
    "You are a helpful assistant who answers questions based on the images provided.\n"
    "Please provide a detailed answer and explanation based on the images. "
    "If you're unable to find the answer from the images, please say 'Unable to find the answer.'\n"
    "USER:\n"
)
for _ in image_arr:
    text += "<image>\n"
text += f"{search_query}\nASSISTANT:"

# Split and tokenize the text
text_chunks = text.split('<image>')
tokenized_chunks = [tokenizer(chunk).input_ids for chunk in text_chunks]

# Build input_ids with placeholders for images
input_ids = tokenized_chunks[0]
for idx in range(1, len(tokenized_chunks)):
    input_ids += [-200]  # Image placeholder
    input_ids += tokenized_chunks[idx]
input_ids = torch.tensor(input_ids, dtype=torch.long).unsqueeze(0).to(device)
image_tensor = model.process_images(image_arr, model.config).to(dtype=model.dtype, device=device)


# generate
output_ids = model.generate(
    input_ids,
    images=image_tensor,
    max_new_tokens=1000,
    use_cache=True,
    repetition_penalty=1.0 
)[0]

print(tokenizer.decode(output_ids[input_ids.shape[1]:], skip_special_tokens=True).strip())

[<PIL.PpmImagePlugin.PpmImageFile image mode=RGB size=2339x1653 at 0x7F4B416E78B0>, <PIL.PpmImagePlugin.PpmImageFile image mode=RGB size=1701x2407 at 0x7F4B416E7940>]
The patient does not have communication issues.
